In [25]:
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

In [26]:
model_name_list = ['Prior', 'LR', 'SVC', 'RF']#, 'INN', 'NBC']

In [27]:
round_precision = 4
# dtype_dict = {'binary_crossentropy': float, 'brier_score': float, 'accuracy': float, 'TN': int, 'TP': int, 'FN': int, 'FP': int}

In [28]:
with open('../data/data_test.pt', 'rb') as file:
    X_test, y_test = pickle.load(file)

print(f'{X_test.shape = }')
print(f'{y_test.shape = }')

X_test.shape = (578417, 33)
y_test.shape = (578417, 2)


In [29]:
y_pred = {}
for model_name in model_name_list:
    with open(f'./predictions/{model_name}.pt', 'rb') as file:
        y_pred[model_name] = pickle.load(file)

        if model_name == 'NBC':
            for y_label in ['hosp', 'death']:
                y_pred[model_name][y_label] = np.concatenate([1 - y_pred[model_name][y_label].reshape(-1, 1), y_pred[model_name][y_label].reshape(-1, 1)], axis=1)

In [30]:
results = {}

for i, y_label in enumerate(['hosp', 'death']):
    results[y_label] = {}
    
    for model_name in model_name_list:
        results[y_label][model_name] = {}

        # average performance over 5 runs
        if model_name in ['INN']:
            log_loss_list = [log_loss(y_test[:, i], y_pred[model_name][y_label][j][:, 1]) for j in range(5)]
            results[y_label][model_name]['binary_crossentropy'] = f'{np.mean(log_loss_list).round(round_precision)} ± {np.std(log_loss_list).round(round_precision)}'
            results[y_label][model_name]['binary_crossentropy_sortable'] = np.mean(log_loss_list).round(round_precision)

            brier_score_list = [brier_score_loss(y_test[:, i], y_pred[model_name][y_label][j][:, 1]) for j in range(5)]
            results[y_label][model_name]['brier_score'] = f'{np.mean(brier_score_list).round(round_precision)} ± {np.std(brier_score_list).round(round_precision)}'

            accuracy_list = [accuracy_score(y_test[:, i], y_pred[model_name][y_label][j][:, 1].round()) for j in range(5)]
            results[y_label][model_name]['accuracy'] = f'{np.mean(accuracy_list).round(round_precision)} ± {np.std(accuracy_list).round(round_precision)}'

            CMs = [confusion_matrix(y_test[:, i], (y_pred[model_name][y_label][j][:, 1] > 0.5).astype(int)) for j in range(5)]
            results[y_label][model_name]['TN'] = f'{np.mean([CMs[j][0, 0] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][0, 0] for j in range(5)]).round(0)}'
            results[y_label][model_name]['TP'] = f'{np.mean([CMs[j][1, 1] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][1, 1] for j in range(5)]).round(0)}'
            results[y_label][model_name]['FP'] = f'{np.mean([CMs[j][0, 1] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][0, 1] for j in range(5)]).round(0)}'
            results[y_label][model_name]['FN'] = f'{np.mean([CMs[j][1, 0] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][1, 0] for j in range(5)]).round(0)}'
        # training is deterministic, no need to average
        else:
            results[y_label][model_name]['binary_crossentropy'] = log_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['binary_crossentropy_sortable'] = log_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['brier_score'] = brier_score_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['accuracy'] = accuracy_score(y_test[:, i], y_pred[model_name][y_label][:, 1].round()).round(round_precision)

            CM = confusion_matrix(y_test[:, i], (y_pred[model_name][y_label][:, 1] > 0.5).astype(int))
            results[y_label][model_name]['TN'] = CM[0, 0]
            results[y_label][model_name]['TP'] = CM[1, 1]
            results[y_label][model_name]['FP'] = CM[0, 1]
            results[y_label][model_name]['FN'] = CM[1, 0]

In [31]:
for y_label in ['hosp', 'death']:
    results[y_label]['Prior']['CPU'] = '-'
    results[y_label]['SVC']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    results[y_label]['RF']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    results[y_label]['LR']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    results[y_label]['INN']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'

    results[y_label]['Prior']['GPU'] = '-'
    results[y_label]['SVC']['GPU'] = '-'
    results[y_label]['RF']['GPU'] = '-'
    results[y_label]['LR']['GPU'] = '-'
    results[y_label]['INN']['GPU'] = 'GTX980Ti@1353/7010MHz'

    results[y_label]['Prior']['Time (HH:mm)'] = '-'
    results[y_label]['SVC']['Time (HH:mm)'] = '00:45'
    results[y_label]['RF']['Time (HH:mm)'] = '02:00'
    results[y_label]['LR']['Time (HH:mm)'] = '03:00'
    results[y_label]['INN']['Time (HH:mm)'] = '26:00'

KeyError: 'INN'

# Results

## Hospitalization

In [32]:
pd.DataFrame(results['hosp']).T.sort_values('binary_crossentropy_sortable').drop(columns='binary_crossentropy_sortable')

,binary_crossentropy,brier_score,accuracy,TN,TP,FP,FN,CPU
SVC,0.208,0.0558,0.9342,538632,1736,390,37659,i7-6700k(4C/8T)@4.5GHz
Prior,0.2487,0.0635,0.9319,539022,0,0,39395,-
RF,0.3544,0.0529,0.9371,535926,6102,3096,33293,i7-6700k(4C/8T)@4.5GHz
LR,0.391,0.1039,0.9274,529932,6520,9090,32875,i7-6700k(4C/8T)@4.5GHz


## Fatality

In [33]:
pd.DataFrame(results['death']).T.sort_values('binary_crossentropy_sortable').drop(columns='binary_crossentropy_sortable')

,binary_crossentropy,brier_score,accuracy,TN,TP,FP,FN
RF,0.0346,0.0087,0.9895,570438.0,1928.0,261.0,5790.0
SVC,0.0416,0.0105,0.9883,570466.0,1204.0,233.0,6514.0
LR,0.0513,0.0106,0.9884,570686.0,1022.0,13.0,6696.0
Prior,0.0709,0.0132,0.9867,570699.0,0.0,0.0,7718.0
